In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
from google.colab import drive
drive.mount('/contents/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /contents/


In [3]:
import numpy as np
import pandas as pd

In [79]:
yelp_reviews=pd.read_csv("/contents/My Drive/review_data_mexican.csv")

In [81]:
yelp_reviews.head()

,cuisine,useful,text
0,Mexican,0,Great basics. Beans are made right. Tortillas ...
1,Mexican,0,Automatic star awarded for having a salsa bar ...
2,Mexican,0,Got the #7 Enchilada and Burrito great food an...
3,Mexican,0,good food for the price. I like their new loca...
4,Mexican,1,One of the better breakfast burritos I have ha...


In [82]:
yelp_reviews["labels"]= yelp_reviews["useful"].apply(lambda x: 1 if x >= 1  else 0)

#yelp_reviews["labels"]= yelp_reviews["stars"].apply(lambda x: 1 if x > 3  else 0)

In [58]:
texts = yelp_reviews.loc[:5000, "text"]
labels = yelp_reviews.loc[:5000, "labels"]

In [59]:
texts[0:6]

0    Great basics. Beans are made right. Tortillas ...
1    Automatic star awarded for having a salsa bar ...
2    Got the #7 Enchilada and Burrito great food an...
3    good food for the price. I like their new loca...
4    One of the better breakfast burritos I have ha...
5    Amado's Bros is a new favorite Friday breakfas...
Name: text, dtype: object

In [60]:
labels[0:6]

0    0
1    0
2    0
3    0
4    1
5    0
Name: labels, dtype: int64

In [61]:
max_length = 128 #512
batch_size = 6


In [11]:
!pip install transformers

     |████████████████████████████████| 778kB 8.9MB/s 
     |████████████████████████████████| 890kB 22.8MB/s 
     |████████████████████████████████| 1.1MB 49.4MB/s 
     |████████████████████████████████| 3.0MB 29.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=320d10478d5f87a7f04a80975d06434f0da5bcbdf685f7e9d1ec6352f2c1a7ba
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [12]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [13]:
import tensorflow as tf

In [14]:
def convert_example_to_feature(review):
  
  # combine step for tokenization, WordPiece vector mapping, adding special tokens as well as truncating reviews longer than the max length
  
  return tokenizer.encode_plus(review, 
                add_special_tokens = True, # add [CLS], [SEP]
                max_length = max_length, # max length of the text that can go to BERT
                pad_to_max_length = True, # add [PAD] tokens
                return_attention_mask = True, # add attention mask to not focus on pad tokens
                truncation=True
              )

In [15]:
convert_example_to_feature("this is just a test")

{'input_ids': [101, 2023, 2003, 2074, 1037, 3231, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

### Create x_train

In [16]:
def encode_examples(texts):

  # prepare list, so that we can build up final TensorFlow dataset from slices.
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    
    for review in texts:

        bert_input = convert_example_to_feature(review)

        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        
        bert_inputs = np.array([input_ids_list, attention_mask_list, token_type_ids_list])
        

    #return map_example_to_dict(input_ids_list, attention_mask_list, token_type_ids_list)
    return bert_inputs


In [62]:
x_train_1 = encode_examples(texts)

In [63]:
x_train_1.shape

(3, 5001, 128)

In [64]:
x_train_1[0].shape

(5001, 128)

In [65]:
x_train_1[0][0:3]

array([[  101,  2307, 24078,  1012, 13435,  2024,  2081,  2157,  1012,
        17153, 28345,  3022,  2024,  5171,  1012,  2021,  1996, 26929,
         2113,  2054,  2027,  2024,  2725,  1998,  1996, 12901,  2015,
         2024,  2682,  3671,  1012,  2123,  1005,  1056,  2022,  6015,
         2065,  2017,  2123,  1005,  1056,  3713,  2394,  2021,  4209,
         1037,  2210,  3009,  3084,  1996,  3325,  2488,  1012,  2065,
         1057,  2444,  2182,  1057,  2323,  3713,  3009,  4312,  2015,
         1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [49]:
# x_train_1 = [x_train_1[0], x_train_1[1], x_train_1[2]]

### Create y_train

In [66]:
labels.shape[0]

5001

In [67]:
y_train_1 = np.array(labels)
y_train_1.shape

(5001,)

### Validation Dataset

In [68]:
len(x_train_1[0])

5001

In [71]:
numSentences = len(x_train_1[0])
np.random.seed(0)
training_examples = np.random.binomial(1, 0.7, numSentences)

trainSentence_ids = []
trainMasks = []
trainSequence_ids = []

testSentence_ids = []
testMasks = []
testSequence_ids = []

Labels_train =[]
Labels_test = []


for example in range(numSentences):
    if training_examples[example] == 1:
        trainSentence_ids.append(x_train_1[0][example])
        trainMasks.append(x_train_1[1][example])
        trainSequence_ids.append(x_train_1[2][example])
        Labels_train.append(y_train_1[example])
    else:
        testSentence_ids.append(x_train_1[0][example])
        testMasks.append(x_train_1[1][example])
        testSequence_ids.append(x_train_1[2][example])
        Labels_test.append(y_train_1[example])
        
X_train = np.array([trainSentence_ids,trainMasks,trainSequence_ids])
X_test = np.array([testSentence_ids,testMasks,testSequence_ids])

y_train = np.array(Labels_train)
y_test = np.array(Labels_test)

X_train = [X_train[0], X_train[1], X_train[2]]
X_test = [X_test[0], X_test[1], X_test[2]]

# **Model**

In [44]:
from transformers import TFBertModel

In [45]:
def classifier_model(max_input_length, train_layers, optimizer):
    """
    Implementation of NER model
    
    variables:
        max_input_length: number of tokens (max_length + 1)
        train_layers: number of layers to be retrained
        optimizer: optimizer to be used
    
    returns: model
    """
    
    in_id = tf.keras.layers.Input(shape=(max_length,), dtype='int32', name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_length,), dtype='int32', name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_length,), dtype='int32', name="segment_ids")
    
    
    bert_inputs = [in_id, in_mask, in_segment]
       
    # Note: Bert layer from Hugging Face returns two values: sequence ouput, and pooled output. Here, we only want
    # the former. (See https://huggingface.co/transformers/model_doc/bert.html#tfbertmodel) 
    
    bert_layer = TFBertModel.from_pretrained('bert-base-cased')
    
    # Freeze layers, i.e. only train number of layers specified, starting from the top
    
    retrain_layers = []
    
    for retrain_layer_number in range(train_layers):
        
        layer_code = '_' + str(11 - retrain_layer_number)
        retrain_layers.append(layer_code)
    
    for w in bert_layer.weights:
        if not any([x in w.name for x in retrain_layers]):
            w._trainable = False
            
    # End of freezing section
    
    bert_sequence = bert_layer(bert_inputs)[0]
    
    print('Let us check the shape of the BERT layer output:', bert_sequence)
    
    dense = tf.keras.layers.Dense(256, activation='relu', name='dense')(bert_sequence)
    
    dense = tf.keras.layers.Dropout(rate=0.1)(dense)
    
    pred = tf.keras.layers.Dense(1, activation='softmax', name='classifier')(dense)
     # sigmoid
    print('pred: ', pred)
    
    ## Prepare for multipe loss functions, although not used here
      
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
       
    model.summary()
    
    return model

In [70]:
X_train[0][0]

array([  101,  2307, 24078,  1012, 13435,  2024,  2081,  2157,  1012,
       17153, 28345,  3022,  2024,  5171,  1012,  2021,  1996, 26929,
        2113,  2054,  2027,  2024,  2725,  1998,  1996, 12901,  2015,
        2024,  2682,  3671,  1012,  2123,  1005,  1056,  2022,  6015,
        2065,  2017,  2123,  1005,  1056,  3713,  2394,  2021,  4209,
        1037,  2210,  3009,  3084,  1996,  3325,  2488,  1012,  2065,
        1057,  2444,  2182,  1057,  2323,  3713,  3009,  4312,  2015,
        1012,   102,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [72]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

model = classifier_model(max_length + 1, train_layers=0, optimizer = 'adam')

model.fit(
    X_train, 
    y_train,
    validation_data=(X_test, y_test),
    epochs=2,
    batch_size=16
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


Let us check the shape of the BERT layer output: Tensor("tf_bert_model_3/Identity:0", shape=(None, 128, 768), dtype=float32)
pred:  Tensor("classifier_3/Identity:0", shape=(None, 128, 1), dtype=float32)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
_______________________________________________________________________________________

In [84]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

model = classifier_model(max_length + 1, train_layers=1, optimizer = 'adam')

model.fit(
    X_train, 
    y_train,
    validation_data=(X_test, y_test),
    epochs=2,
    batch_size=16
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


Let us check the shape of the BERT layer output: Tensor("tf_bert_model_4/Identity:0", shape=(None, 128, 768), dtype=float32)
pred:  Tensor("classifier_4/Identity:0", shape=(None, 128, 1), dtype=float32)
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
_______________________________________________________________________________________

## TFBertForSequenceClassification

In [51]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf

# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
learning_rate = 2e-5

# we will do just 1 epoch for illustration, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 1


# model initialization
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# choosing Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier', 'dropout_189']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [73]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=number_of_epochs)

110/110 [==============================] - 56s 512ms/step - loss: 8.7194 - accuracy: 0.4282 - val_loss: 9.2071 - val_accuracy: 0.3962


In [83]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Epoch 1/3
110/110 [==============================] - 56s 512ms/step - loss: 8.7194 - accuracy: 0.4282 - val_loss: 9.2071 - val_accuracy: 0.3962
Epoch 2/3
110/110 [==============================] - 56s 513ms/step - loss: 8.7194 - accuracy: 0.4282 - val_loss: 9.2071 - val_accuracy: 0.3962
Epoch 3/3
110/110 [==============================] - 56s 513ms/step - loss: 8.7194 - accuracy: 0.4282 - val_loss: 9.2071 - val_accuracy: 0.3962
